# 4colab

In [1]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# !ls

# data_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# result_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/result'

In [2]:
data_dir = 'data'
result_dir = 'result'

# Import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [4]:
af = 'vv'
dt = 'v'
pf = '__5__'

Xtrain = np.load(f'{data_dir}/{af}3.tr.X.{pf}.npy', allow_pickle=True)
y_train = np.load(f'{data_dir}/{af}3.tr.y.npy', allow_pickle=True)

Xtest = np.load(f'{data_dir}/{af}3.t.X.{pf}.npy', allow_pickle=True)
y_test = np.load(f'{data_dir}/{af}3.t.y.npy', allow_pickle=True)

# Xval = np.load(f'{data_dir}/{af}3.v.X.{pf}.npy', allow_pickle=True)
# y_val = np.load(f'{data_dir}/{af}3.v.y.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}3.fts_cols.{pf}.txt').readlines()]

In [5]:
Xtrain.shape

(1529165, 65)

In [6]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

# Test and evaluate

In [7]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")
    return roc_auc

In [8]:
import pickle

def save_result(model, Xtr, Xt, model_name, selected_fts_names, roc_auc, expname='', config=None):
    ra = '{:.2f}'.format(round(roc_auc * 100, 2))
    # if ra <= 70: #? not good enough to even bother
    #     return
    
    #? save model
    pickle.dump(model, open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.model.pkl','wb'))

    #? save transformed X as well
    np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.data.tr.X.npy', Xtr)
    np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.data.t.X.npy', Xt)
    # np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.data.v.X.npy', Xv)

    #? save fts
    open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.fts.txt', 'w').write('\n'.join(selected_fts_names))
    
    #? save config
    if config is not None:
        open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.config.txt', 'w').write(config)

## Exp 01: Run on original features (0-11)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`

Using only numeric features from original data, the result is not good

```
CPU times: total: 34min 16s
Wall time: 3min 41s

CPU times: total: 18min 29s
Wall time: 1min 54s

[[748520  13060]
 [  2682    280]]
              precision    recall  f1-score   support

           0     0.9964    0.9829    0.9896    761580
           1     0.0210    0.0945    0.0344      2962

    accuracy                         0.9794    764542
   macro avg     0.5087    0.5387    0.5120    764542
weighted avg     0.9927    0.9794    0.9859    764542

0.5386910814556447
```

In [9]:
""" select features """
selected_fts_names__01 = fts_names[0:12]
', '.join(selected_fts_names__01)

'Dur, sTos, dTos, Sport, Dport, TotPkts, TotBytes, SrcBytes, PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt'

In [10]:
X_train__01 = Xtrain[:,0:12]
X_test__01 = Xtest[:,0:12]
# X_val__01 = Xval[:,0:12]
print(X_train__01.shape)

(1529165, 12)


In [57]:
""" train """
from sklearn.cluster import DBSCAN

eps = 0.0002 #0.0375
min_samples = 3
model__01 = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=10)

%time model__01.fit(X_train__01)

CPU times: total: 36min 55s
Wall time: 4min 5s


DBSCAN(eps=0.00015, min_samples=3, n_jobs=10)

In [59]:
np.unique(model__01.labels_)

array([0, 1], dtype=int64)

In [58]:
%%time
y_pred__01 = model__01.labels_
# y_pred__01[y_pred__01 == 2]
y_pred__01[y_pred__01 != -1] = 0
y_pred__01[y_pred__01 == -1] = 1

roc_auc__01 = evaluate(y_train, y_pred__01)

[[1518139    1075]
 [   7840    2111]]
              precision    recall  f1-score   support

           0     0.9949    0.9993    0.9971   1519214
           1     0.6626    0.2121    0.3214      9951

    accuracy                         0.9942   1529165
   macro avg     0.8287    0.6057    0.6592   1529165
weighted avg     0.9927    0.9942    0.9927   1529165

0.605715940360894
CPU times: total: 1.88 s
Wall time: 1.88 s


In [52]:
df_cluster__01 = pd.DataFrame(model__01.labels_, columns=['clusters'])
df_pred__01 = pd.DataFrame(y_pred__01, columns=['Label_Pred'])
df_ft__01 = pd.DataFrame(X_train__01)
df_y = pd.DataFrame(y_train, columns=['Label'])
df__01 = pd.concat([df_ft__01, df_y, df_cluster__01, df_pred__01], axis=1)
df__01

,0,1,2,3,4,5,6,7,8,9,10,11,Label,clusters,Label_Pred
0,6.583431e-01,0.25,0.25,0.182050,2.478763e-05,9.796596e-07,0.647358,1.633007e-07,2.464168e-09,0.021701,2.411215e-10,0.170260,0,0,0
1,1.641915e-01,0.25,0.25,0.781238,2.478577e-05,7.347447e-07,0.647358,1.610789e-07,7.410255e-09,0.021701,9.536480e-10,0.176145,0,0,0
2,5.136541e-05,0.25,0.25,0.024002,1.484178e-07,2.938979e-06,0.647360,8.320560e-07,9.474868e-05,0.021826,1.574642e-05,0.292437,1,0,0
3,5.174788e-01,0.25,0.25,0.519325,2.478763e-05,2.694064e-06,0.647358,5.798841e-07,8.621102e-09,0.021701,1.089303e-09,0.186029,0,0,0
4,0.000000e+00,0.25,0.00,0.157148,1.276393e-05,0.000000e+00,0.647358,6.554246e-08,0.000000e+00,0.021701,0.000000e+00,0.151485,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529160,6.921464e-05,0.25,0.25,0.824359,2.040745e-07,4.163553e-06,0.647359,6.243197e-07,9.961245e-05,0.021724,8.768175e-06,0.175650,0,0,0
1529161,1.918196e-07,0.25,0.25,0.766239,9.832680e-08,4.898298e-07,0.647358,8.553846e-08,4.228630e-03,0.023313,4.334796e-04,0.192398,0,0,0
1529162,2.495132e-07,0.25,0.25,0.856158,9.832680e-08,2.449149e-07,0.647358,8.553846e-08,1.625433e-03,0.022571,3.332484e-04,0.208931,0,0,0
1529163,1.647897e-07,0.25,0.25,0.834063,9.832680e-08,4.898298e-07,0.647358,9.331469e-08,4.922238e-03,0.023480,5.504528e-04,0.190296,0,0,0


In [47]:
print(df__01['clusters'].value_counts(), '\n')
print(df__01.loc[df__01['Label'] == 1]['clusters'].value_counts())

0    1525429
1       3736
Name: clusters, dtype: int64 

0    7525
1    2426
Name: clusters, dtype: int64


In [51]:
print(df__01.loc[df__01['clusters'] == 1]['Label'].value_counts(), '\n')
print(df__01.loc[df__01['Label_Pred'] == 1]['Label'].value_counts())

1    2426
0    1310
Name: Label, dtype: int64 

1    2426
0    1310
Name: Label, dtype: int64


In [31]:
df__01.loc[df__01['Label'] == 1].to_csv('dbscan.tr.csv')

In [12]:
%%time
""" predict """
y_pred__01 = model__01.predict(X_test__01)

CPU times: total: 18min 29s
Wall time: 1min 54s


In [13]:
""" evaluate """
#? relabel to use classification metrics
y_pred__01[y_pred__01 == 1] = 0
y_pred__01[y_pred__01 == -1] = 1

roc_auc__01 = evaluate(y_test, y_pred__01)

[[748520  13060]
 [  2682    280]]
              precision    recall  f1-score   support

           0     0.9964    0.9829    0.9896    761580
           1     0.0210    0.0945    0.0344      2962

    accuracy                         0.9794    764542
   macro avg     0.5087    0.5387    0.5120    764542
weighted avg     0.9927    0.9794    0.9859    764542

0.5386910814556447


### Save model ?

In [15]:
save_result(model__01, X_train__01, X_test__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01__fts-0-11', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

## Exp 02: Run on all features

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `DstBytes`, `DstBytesPerSec`, `SportHex`, `DportHex`, `sTos_-1`, `sTos_0`, `sTos_1`, `sTos_2`, `sTos_3`, `dTos_-1`, `dTos_0`, `dTos_1`, `dTos_2`, `dTos_3`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The result increase, yet running time is much higher

```
CPU times: total: 8h 8min 53s
Wall time: 42min 7s

CPU times: total: 4h 59min 53s
Wall time: 22min

[[748312  13268]
 [  2727    235]]
              precision    recall  f1-score   support

           0     0.9964    0.9826    0.9894    761580
           1     0.0174    0.0793    0.0285      2962

    accuracy                         0.9791    764542
   macro avg     0.5069    0.5310    0.5090    764542
weighted avg     0.9926    0.9791    0.9857    764542

0.5309583044766079
```

In [16]:
""" select features """
selected_fts_names__02 = fts_names[:]
', '.join(selected_fts_names__02)

'Dur, sTos, dTos, Sport, Dport, TotPkts, TotBytes, SrcBytes, PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt, DstBytes, DstBytesPerSec, SportHex, DportHex, sTos_-1, sTos_0, sTos_1, sTos_2, sTos_3, dTos_-1, dTos_0, dTos_1, dTos_2, dTos_3, State_CON, State_alltcp, State_INT, State_S_, State_URP, State_ECO, State_RED, State_REQ, State_ECR, State_URH, State_TXD, State_URFIL, State_R_, State_URN, State_RSP, State_URHPRO, State_A_, State_other, Flag_nan, Flag_S, Flag_A, Flag_P, Flag_R, Flag_F, Proto_udp, Proto_tcp, Proto_icmp, Proto_rtp, Proto_rtcp, Proto_igmp, Proto_arp, Proto_other, Service_80, Service_443, Service_21, Service_22, Service_25, Service_6667, Service_other'

In [17]:
X_train__02 = Xtrain[:,:]
X_test__02 = Xtest[:,:]
# X_val__02 = Xval[:,:]
print(X_train__02.shape)

(1529165, 65)


In [18]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 30
contamination = 0.01
model__02 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=4)

%time model__02.fit(X_train__02)

CPU times: total: 8h 8min 53s
Wall time: 42min 7s


LocalOutlierFactor(contamination=0.01, n_jobs=4, n_neighbors=30, novelty=True)

In [19]:
%%time
""" predict """
y_pred__02 = model__02.predict(X_test__02)

CPU times: total: 4h 59min 53s
Wall time: 22min


In [20]:
""" evaluate """
#? relabel to use classification metrics
y_pred__02[y_pred__02 == 1] = 0
y_pred__02[y_pred__02 == -1] = 1

roc_auc__02 = evaluate(y_test, y_pred__02)

[[748312  13268]
 [  2727    235]]
              precision    recall  f1-score   support

           0     0.9964    0.9826    0.9894    761580
           1     0.0174    0.0793    0.0285      2962

    accuracy                         0.9791    764542
   macro avg     0.5069    0.5310    0.5090    764542
weighted avg     0.9926    0.9791    0.9857    764542

0.5309583044766079


### Save model ?

In [21]:
save_result(model__02, X_train__02, X_test__02, 'lof', selected_fts_names__02, roc_auc__02, 'exp02__fts-all', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

## Exp 03: Run on generated features (8-65)

`PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `DstBytes`, `DstBytesPerSec`, `SportHex`, `DportHex`, `sTos_-1`, `sTos_0`, `sTos_1`, `sTos_2`, `sTos_3`, `dTos_-1`, `dTos_0`, `dTos_1`, `dTos_2`, `dTos_3`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The result is not much different from Exp01 

```
CPU times: total: 6h 54min 21s
Wall time: 29min 12s

CPU times: total: 3h 32min 6s
Wall time: 44min 9s

[[748227  13353]
 [  2726    236]]
              precision    recall  f1-score   support

           0     0.9964    0.9825    0.9894    761580
           1     0.0174    0.0797    0.0285      2962

    accuracy                         0.9790    764542
   macro avg     0.5069    0.5311    0.5089    764542
weighted avg     0.9926    0.9790    0.9856    764542

0.5310713043012909
```

In [9]:
""" select features """
selected_fts_names__03 = fts_names[8:65]
', '.join(selected_fts_names__03)

'PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt, DstBytes, DstBytesPerSec, SportHex, DportHex, sTos_-1, sTos_0, sTos_1, sTos_2, sTos_3, dTos_-1, dTos_0, dTos_1, dTos_2, dTos_3, State_CON, State_alltcp, State_INT, State_S_, State_URP, State_ECO, State_RED, State_REQ, State_ECR, State_URH, State_TXD, State_URFIL, State_R_, State_URN, State_RSP, State_URHPRO, State_A_, State_other, Flag_nan, Flag_S, Flag_A, Flag_P, Flag_R, Flag_F, Proto_udp, Proto_tcp, Proto_icmp, Proto_rtp, Proto_rtcp, Proto_igmp, Proto_arp, Proto_other, Service_80, Service_443, Service_21, Service_22, Service_25, Service_6667, Service_other'

In [10]:
# X_train__03 = np.concatenate((Xtrain[:,8:13], Xtrain[:,17:65]))
X_train__03 = Xtrain[:,8:65]
X_test__03 = Xtest[:,8:65]
# X_val__03 = Xval[:,8:65]
print(X_train__03.shape)

(1529165, 57)


In [11]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 30
contamination = 0.01
model__03 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=4)

%time model__03.fit(X_train__03)

CPU times: total: 6h 54min 21s
Wall time: 29min 12s


LocalOutlierFactor(contamination=0.01, n_jobs=4, n_neighbors=30, novelty=True)

In [12]:
%%time
""" predict """
y_pred__03 = model__03.predict(X_test__03)

CPU times: total: 3h 32min 6s
Wall time: 44min 9s


In [13]:
""" evaluate """
#? relabel to use classification metrics
y_pred__03[y_pred__03 == 1] = 0
y_pred__03[y_pred__03 == -1] = 1

roc_auc__03 = evaluate(y_test, y_pred__03)

[[748227  13353]
 [  2726    236]]
              precision    recall  f1-score   support

           0     0.9964    0.9825    0.9894    761580
           1     0.0174    0.0797    0.0285      2962

    accuracy                         0.9790    764542
   macro avg     0.5069    0.5311    0.5089    764542
weighted avg     0.9926    0.9790    0.9856    764542

0.5310713043012909


### Save model ?

In [14]:
save_result(model__03, X_train__03, X_test__03, 'lof', selected_fts_names__03, roc_auc__03, 'exp03__fts-8-65', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

## Exp 04: Run on chosen set of features

`sTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The overall ROC_AUC end execution time does not change much from experiment 02 or 03.  
Recall for minority class increases.  

```
CPU times: total: 7h 17min 18s
Wall time: 27min 47s

CPU times: total: 3h 15min 41s
Wall time: 13min 47s

[[744296  17284]
 [  1711   1251]]
              precision    recall  f1-score   support

           0     0.9977    0.9773    0.9874    761580
           1     0.0675    0.4223    0.1164      2962

    accuracy                         0.9752    764542
   macro avg     0.5326    0.6998    0.5519    764542
weighted avg     0.9941    0.9752    0.9840    764542

0.6998274199809809
```

In [11]:
""" select features """
selected_fts_names__04 = ['sTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__04 = [fts_names.index(s) for s in selected_fts_names__04]
len(idx__selected_fts_names__04)

49

In [12]:
X_train__04 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
X_test__04 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)

print(X_train__04.shape)

(1529165, 49)


In [14]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 30
contamination = 0.01
model__04 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=10)

%time model__04.fit(X_train__04)

CPU times: total: 7h 17min 18s
Wall time: 27min 47s


LocalOutlierFactor(contamination=0.01, n_jobs=10, n_neighbors=30, novelty=True)

In [15]:
%%time
""" predict """
y_pred__04 = model__04.predict(X_test__04)

CPU times: total: 3h 15min 41s
Wall time: 13min 47s


In [16]:
""" evaluate """
#? relabel to use classification metrics
y_pred__04[y_pred__04 == 1] = 0
y_pred__04[y_pred__04 == -1] = 1

roc_auc__04 = evaluate(y_test, y_pred__04)

[[744296  17284]
 [  1711   1251]]
              precision    recall  f1-score   support

           0     0.9977    0.9773    0.9874    761580
           1     0.0675    0.4223    0.1164      2962

    accuracy                         0.9752    764542
   macro avg     0.5326    0.6998    0.5519    764542
weighted avg     0.9941    0.9752    0.9840    764542

0.6998274199809809


### Save model ?

In [14]:
save_result(model__04, X_train__04, X_test__04, 'lof', selected_fts_names__04, roc_auc__04, 'exp04__play', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

## Exp 05: Run on numeric features chosen from MI + one-hot features

**Chosen features from MI: `dTos`, `Sport`, `BytesPerPkt`, `Dur`, `BytesPerSec`, `DstBytesPerSec`, `PktsPerSec`, `SrcBytesPerSec`, `TotBytes`, `DstBytes`**
```
CPU times: total: 6h 16min 9s
Wall time: 31min 7s

CPU times: total: 3h 30min 25s
Wall time: 14min 11s

[[751188  10392]
 [  1586   1376]]
              precision    recall  f1-score   support

           0     0.9979    0.9864    0.9921    761580
           1     0.1169    0.4646    0.1868      2962

    accuracy                         0.9843    764542
   macro avg     0.5574    0.7255    0.5895    764542
weighted avg     0.9945    0.9843    0.9890    764542

0.7254528313760588
```

In [9]:
""" select features """
selected_fts_names__05 = ['dTos', 'Sport', 'BytesPerPkt', 'Dur', 'BytesPerSec', 'DstBytesPerSec', 'PktsPerSec', 'SrcBytesPerSec', 'TotBytes', 'DstBytes', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__05 = [fts_names.index(s) for s in selected_fts_names__05]
len(idx__selected_fts_names__05)

49

In [10]:
X_train__05 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)
X_test__05 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)

print(X_train__05.shape)

(1529165, 49)


In [11]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 30
contamination = 0.01
model__05 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=10)

%time model__05.fit(X_train__05)

CPU times: total: 6h 16min 9s
Wall time: 31min 7s


LocalOutlierFactor(contamination=0.01, n_jobs=10, n_neighbors=30, novelty=True)

In [12]:
%%time
""" predict """
y_pred__05 = model__05.predict(X_test__05)

CPU times: total: 3h 30min 25s
Wall time: 14min 11s


In [13]:
""" evaluate """
#? relabel to use classification metrics
y_pred__05[y_pred__05 == 1] = 0
y_pred__05[y_pred__05 == -1] = 1

roc_auc__05 = evaluate(y_test, y_pred__05)

[[751188  10392]
 [  1586   1376]]
              precision    recall  f1-score   support

           0     0.9979    0.9864    0.9921    761580
           1     0.1169    0.4646    0.1868      2962

    accuracy                         0.9843    764542
   macro avg     0.5574    0.7255    0.5895    764542
weighted avg     0.9945    0.9843    0.9890    764542

0.7254528313760588


### Save model ?

In [18]:
save_result(model__05, X_train__05, X_test__05, 'lof', selected_fts_names__05, roc_auc__05, 'exp05__mi', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

## Exp 06: Run on features chosen from MI

`State_S_` ,` dTos` ,` Proto_tcp` ,` Proto_udp` ,` Service_80` ,` Service_other` ,` State_CON` ,` Service_25` ,` Sport` ,` Flag_S` ,` Flag_R` ,` BytesPerPkt` ,` Dur` ,` Service_6667` ,` BytesPerSec` ,` State_alltcp` ,` DstBytesPerSec` ,` Flag_A` ,` PktsPerSec` ,` SrcBytesPerSec`

Result is slightly better than Exp05

```
CPU times: total: 6h 9min 8s
Wall time: 24min 7s

CPU times: total: 2h 57min 22s
Wall time: 11min 15s

[[752515   9065]
 [  1550   1412]]
              precision    recall  f1-score   support

           0     0.9979    0.9881    0.9930    761580
           1     0.1348    0.4767    0.2101      2962

    accuracy                         0.9861    764542
   macro avg     0.5664    0.7324    0.6016    764542
weighted avg     0.9946    0.9861    0.9900    764542

0.732401021498378
```

In [20]:
""" select features """
# selected_fts_names__06 = fts_names[29:30] + fts_names[2:3] + fts_names[51:52] + fts_names[50:51] + fts_names[58:59] + fts_names[64:65] + fts_names[26:27] + fts_names[62:63] + fts_names[3:4] + fts_names[45:46] + fts_names[48:49] + fts_names[26:65]
selected_fts_names__06 = ['State_S_','dTos','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Sport','Flag_S','Flag_R','BytesPerPkt','Dur','Service_6667','BytesPerSec','State_alltcp','DstBytesPerSec','Flag_A','PktsPerSec','SrcBytesPerSec']
idx__selected_fts_names__06 = [fts_names.index(s) for s in selected_fts_names__06]
len(idx__selected_fts_names__06)
# ', '.join(selected_fts_names__05)

20

In [21]:
X_train__06 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
X_test__06 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)

print(X_train__06.shape)

(1529165, 20)


In [23]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 30
contamination = 0.01
model__06 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=4)

%time model__06.fit(X_train__06)

CPU times: total: 6h 9min 8s
Wall time: 24min 7s


LocalOutlierFactor(contamination=0.01, n_jobs=4, n_neighbors=30, novelty=True)

In [24]:
%%time
""" predict """
y_pred__06 = model__06.predict(X_test__06)

CPU times: total: 2h 57min 22s
Wall time: 11min 15s


In [25]:
""" evaluate """
#? relabel to use classification metrics
y_pred__06[y_pred__06 == 1] = 0
y_pred__06[y_pred__06 == -1] = 1

roc_auc__06 = evaluate(y_test, y_pred__06)

[[752515   9065]
 [  1550   1412]]
              precision    recall  f1-score   support

           0     0.9979    0.9881    0.9930    761580
           1     0.1348    0.4767    0.2101      2962

    accuracy                         0.9861    764542
   macro avg     0.5664    0.7324    0.6016    764542
weighted avg     0.9946    0.9861    0.9900    764542

0.732401021498378


### Save model ?

In [26]:
save_result(model__06, X_train__06, X_test__06, 'lof', selected_fts_names__06, roc_auc__06, 'exp06__mi', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

## Checking score

The best set of features is of Exp06.  
Use this experiment result to further analyze the scores of outliers

Check `vv2-6.after_predict`